<a href="https://colab.research.google.com/github/LuigiElo/DL-PINNs/blob/main/PINN_LaPlace2D_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Network

In [8]:
import torch
import torch.nn as nn

class SineActivation(nn.Module):
    def forward(self, x):
        return torch.sin(x)

class LinearNN(nn.Module):
    def __init__(
        self,
        num_inputs: int = 2,
        num_layers: int = 3,
        num_neurons: int = 32,
        act: nn.Module = SineActivation(),
    ) -> None:
        """Basic neural network architecture with linear layers

        Args:
            num_inputs (int, optional): the dimensionality of the input tensor
            num_layers (int, optional): the number of hidden layers
            num_neurons (int, optional): the number of neurons for each hidden layer
            act (nn.Module, optional): the non-linear activation function to use for stitching
                linear layers togeter
        """
        super().__init__()

        self.num_inputs = num_inputs
        self.num_neurons = num_neurons
        self.num_layers = num_layers

        layers = []

        # input layer
        layers.append(nn.Linear(self.num_inputs, num_neurons))

        # hidden layers with linear layer and activation
        for _ in range(num_layers):
            layers.extend([nn.Linear(num_neurons, num_neurons), act])

        # output layer
        layers.append(nn.Linear(num_neurons, 1))

        # build the network
        self.network = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x).squeeze()

#network

# Example usage:
"""
model = LinearNN()
input_tensor = torch.randn((10, 2))  # Example input with batch size 10 and 2 features
output = model(input_tensor)

print("Input shape:", input_tensor)
print("Output shape:", output)"""

'\nmodel = LinearNN()\ninput_tensor = torch.randn((10, 2))  # Example input with batch size 10 and 2 features\noutput = model(input_tensor)\n\nprint("Input shape:", input_tensor)\nprint("Output shape:", output)'

#PINN Loss

In [ ]:
import torch.nn.functional as F

def data_loss(predicted_data, target_data): #data has to be in z=0, 0<x<L
        #target data is obtained using behaviour function
        return F.mse_loss(predicted_data, target_data)

def boundary_condition_loss(boundary_conditions):
    # Customize based on your specific boundary conditions
    # Example: Dirichlet boundary condition u(0, t) = g(t)
    loss_bc = F.mse_loss(boundary_conditions['bc1'], 0) + F.mse_loss(boundary_conditions['bc2_0'], boundary_conditions['bc2_0'])
    # instead of 0 it will be an array of zeros with the same length as the number of points we decide to use to calculate the loss
    return loss_bc

def pde_loss(model,input_data): #here input_data has to be in -h(x)<x2<0

    x1, x2 = input_data[:, 0], input_data[:, 1]

    # Forward pass to get the function values
    u = model(input_data)

    # Compute the Hessian matrix
    hessian = compute_hessian(u, input_data)

    # Extract elements corresponding to (0, 0) and (1, 1)
    hessian_00 = hessian[:, 0, 0]
    hessian_11 = hessian[:, 1, 1]

    # Enforce the PDE constraint
    pde_residual = hessian_00 + hessian_11


    return F.mse_loss(pde_residual, torch.zeros_like(pde_residual.size))

def compute_bc1(model, x1, x2):
    # Concatenate x1 and x2 to form the input tensor
    input_tensor = torch.stack([x1, x2], dim=1)

    # Forward pass through the model
    output = model(input_tensor)

    # Compute the gradient with respect to x2
    grad_x2 = torch.autograd.grad(output, x2, create_graph=True)[0]

    return grad_x2

def compute_bc2(model, x1, x2):
    # Concatenate x1 and x2 to form the input tensor
    input_tensor = torch.stack([x1, x2], dim=1)

    # Forward pass through the model
    output = model(input_tensor)

    return output

def compute_hessian(output, input_data):
    # Compute the Hessian matrix
    hessian = autograd.functional.hessian(lambda x: output.sum(), input_data)

    return hessian

#x1 and x2 have to represent the points we want to apply the constraints

boundary_conditions = {
    'bc1': compute_bc1(model, x1, x2),   # at x2=-1 the grad perpendicular to the bottom is null
    'bc2_0': compute_bc2(model, 0, x2),  # at x1 = 0
    'bc2_2': compute_bc2(model, 2, x2),  # at x1 = 2
}

#total_loss = loss_data + loss_bc + loss_pde

#the data we provide to calculate each loss is extremely important. It has to belong to the domain where the constraints/pde/function are applied.

In [7]:
import numpy as np

# General parameters
L = 2.0 # wave/x length
h = 1 # depth
T = 1 # period
H = 0.05 # surface wave amplitude (I chose it arbitrarliy. Check p. 68 HRC)
c = L/T # wave propagation velocity (p. 70 HCR)
k = 2*np.pi/L # wave nr.
w = 2*np.pi/T

t = 0 # we are looking at a snapshot, so t is constant

# Analytical solution (HRC p. 75)
def behaviour_func(x):
    return -H*c/2 * np.cosh(k*(x[:, 1:2]+h))/np.sinh(k*h) * np.sin(w*t - k*x[:, 0:1])
